In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, to_date, split, when, col, sum, mean, round, count, abs
import numpy as np
from datetime import datetime, timedelta
from pyspark.sql.window import Window
from pyspark.sql import functions as F

In [17]:
spark = SparkSession.builder \
    .appName("machungkhoan") \
    .config("spark.cores.max", "2") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

# Đường dẫn tới bảng nguồn machungkhoan
source_table_path = "stock_db.datn_machungkhoan"

# Đọc dữ liệu từ bảng nguồn machungkhoan
df_machungkhoan = spark.read.format("iceberg").load(source_table_path)

In [23]:
df_stock = df_machungkhoan.filter((col('companyname').isNotNull()) & (col('companyname') != "") & (col('categoryid') != 0))

In [25]:
# Lựa chọn các cột cần thiết từ DataFrame
selected_columns = df_stock.select(
    "symbol", "companyname", "categoryname"
)

In [26]:
df_stock_drop_duplicate = selected_columns.dropDuplicates(['symbol'])

In [29]:
# Tiếp theo, ghi dữ liệu vào bảng dim_stock
dim_stock_data = df_stock_drop_duplicate.select(
    col("symbol").alias("stocksymbol"),
    "companyname",
    "categoryname"
).distinct()

In [30]:
dim_stock_data.show()

+-----------+--------------------+--------------------+
|stocksymbol|         companyname|        categoryname|
+-----------+--------------------+--------------------+
|        24H|CTCP Quảng cáo tr...|           Công nghệ|
|        A32|  Công ty cổ phần 32|      Hàng tiêu dùng|
|        AAA|Công ty Cổ phần N...|     Nguyên vật liệu|
|        AAM|Công ty Cổ phần T...|         Nông nghiệp|
|        AAS|Công ty cổ phần C...|           Tài chính|
|        AAT|Công ty Cổ phần T...|      Hàng tiêu dùng|
|        AAV|Công ty Cổ phần A...|Bất động sản và X...|
|        ABB|Ngân hàng Thương ...|           Tài chính|
|        ABC|Công ty cổ phần T...|          Viễn thông|
|        ABF|Công ty cổ phần Q...|           Tài chính|
|        ABI|Công ty Cổ phần B...|           Tài chính|
|        ABR|Công ty Cổ phần Đ...|      Hàng tiêu dùng|
|        ABS|Công ty cổ phần D...|         Nông nghiệp|
|        ABT|Công ty Cổ phần X...|         Nông nghiệp|
|        ACB|Ngân hàng Thương ...|           Tài

In [31]:

# Ghi dữ liệu vào bảng dim_stock
dim_stock_data.write \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", "jdbc:mysql://10.168.6.106:3306/dtm_stock_v2") \
    .option("dbtable", "dim_stock") \
    .option("user", "acc_etl") \
    .option("password", "Vnpt123456") \
    .mode("append") \
    .save()